In [2]:
from __future__ import print_function
import os
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support
from keras.models import model_from_json
from keras.models import load_model

try:
    import cPickle as pickle
except ImportError:
    import pickle
from PIL import Image
from six.moves import range
import keras.backend as K
from keras.datasets import cifar10
from keras import layers
from keras.layers import Input, Dense, Reshape, Flatten, Embedding, Dropout, BatchNormalization , Lambda , dot
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2DTranspose, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.initializers import TruncatedNormal
from keras.utils.generic_utils import Progbar
from Minibatch import MinibatchDiscrimination
import matplotlib.pyplot as plt
from keras.layers.noise import GaussianNoise
import numpy as np
from keras.utils import to_categorical
import tensorflow as tf
from math import log
from keras.initializers import Constant


Using TensorFlow backend.


In [3]:
np.random.seed(1337)
class_num = 10
num_classes = 10
K.set_image_dim_ordering('th')
 # The path to store the generated images
load_weight = False
# Set True if you need to reload weight
load_epoch = 151 # Decide which epoch to reload weight, please check your file name
Path = os.getcwd() +  '/7_class_drop_99'
pi = 0.01
kconst = 0.01
bias_value = -log((1-pi)/pi)
print(bias_value)
if not os.path.exists(Path):
    os.makedirs(Path)

-4.59511985013459


In [4]:
    
def get_data():
    (X_train, y_train),(X_test, y_test) = cifar10.load_data()
    X_temp =  X_train.astype('float32')
    mean_image = np.mean(X_temp , axis = 0)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_test = (X_test.astype(np.float32) - 127.5) / 127.5
    return X_train, y_train, X_test, y_test

In [14]:
# def categorical_focal_loss(gamma, inverse_class_weights):
    
#     def focal_loss_fnc(y_label, y_pred):
#         epsilon = 1.e-9
                           
#         y_true = tf.convert_to_tensor(y_label, tf.float32)
#         y_pred = tf.convert_to_tensor(y_pred, tf.float32)
#         mask = tf.convert_to_tensor(y_pred , tf.float32)
#         alpha = tf.matmul(y_label, inverse_class_weights)

#         model_out = tf.add(y_pred, epsilon)
#         ce = tf.multiply(y_true, -tf.log(model_out))
#         weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
#         fl = tf.multiply(alpha, tf.multiply(weight, ce))
#         reduced_fl = tf.reduce_max(fl, axis=1)

#         return tf.reduce_mean(reduced_fl)

#     return focal_loss_fnc

In [5]:
def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)
    
    

    return categorical_focal_loss_fixed

In [6]:

def get_imbalanced_train_data(drop_list, drop_percentage):
    
    x_train, y_train, x_test, y_test = get_data()

    y_train = np.squeeze(y_train)
    y_test = np.squeeze(y_test)
    
    per_class_count = list()
    for c in range(num_classes):
        per_class_count.append(np.sum(np.array(y_train == c)))
    print ('Per class count before dropping: ')
    print (per_class_count) 

    for class_num in drop_list: 
        all_ids = list(np.arange(len(x_train)))
        mask = [class_num == lc for lc in y_train]
        # mask is a boolean array of index where class_num is present in y_train
        all_ids_c = np.array(all_ids)[mask]
        np.random.shuffle(all_ids_c)
        # all_ids_c is shuffled np array of all indexes of class_num

        other_class_count = np.array(per_class_count)
        other_class_count = np.delete(other_class_count, class_num)
        to_keep = int(np.ceil((1. - drop_percentage) * per_class_count[class_num]))

        to_delete = all_ids_c[to_keep: len(all_ids_c)]

        x_train = np.delete(x_train, to_delete, axis=0)
        y_train = np.delete(y_train, to_delete, axis=0)

    unbalanced_per_class_count = list()
    for c in range(num_classes):
        unbalanced_per_class_count.append(np.sum(np.array(y_train == c)))
    
    print ('Per class count after dropping: ')
    print (unbalanced_per_class_count) 
    
    class_weights = []
    for nums in unbalanced_per_class_count:
        class_weights.append(np.sum(unbalanced_per_class_count)/nums)
        
    class_weights = np.asarray(class_weights, dtype = np.float32)
    inverse_class_weights = class_weights/np.sum(class_weights)
    inverse_class_weights = inverse_class_weights[:, np.newaxis]
    
#     y_train = onehot[y_train.astype(np.int32)]
#     y_test = onehot[y_test.astype(np.int32)]
    
    return x_train, y_train, x_test, y_test, inverse_class_weights



In [7]:
def build_generator(latent_size):
    # we will map a pair of (z, L), where z is a latent vector and L is a
    # label drawn from P_c, to image space (..., 3, 32, 32)
    cnn = Sequential()
    cnn.add(Dense(384 * 4 * 4, input_dim=latent_size, activation='relu',
                  kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(Reshape((384, 4, 4)))

    cnn.add(Conv2DTranspose(192, kernel_size=5, strides=2, padding='same', activation='relu',
                            kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())

    cnn.add(Conv2DTranspose(96, kernel_size=5, strides=2, padding='same', activation='relu',
                            kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())

    cnn.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh',
                            kernel_initializer='glorot_normal', bias_initializer='Zeros'))

    # this is the z space commonly refered to in GAN papers
    latent = Input(shape=(latent_size,))

    # this will be our label
    image_class = Input(shape=(10,)) 
    print(type(image_class))
#     image_class = Reshape((10,1), input_shape=(10,))
    
    cls_emb = Dense(110,)(image_class)
   
    
    # print(cls.output_shape)
#     cls = Dense(110 , input_dim = num_classes , kernel_initializer = 'glorot_normal')(image_class)
    # hadamard product between z-space and a class conditional embedding
    h = layers.multiply([latent, cls_emb])
   
    fake_image = cnn(h)
  
    return Model([latent, image_class], fake_image)



In [8]:

def build_discriminator():
    # build a relatively standard conv net, with LeakyReLUs as suggested in
    # the reference paper
    cnn = Sequential()

    cnn.add(GaussianNoise(0.05, input_shape=(3, 32, 32)))  # Add this layer to prevent D from overfitting!

    cnn.add(Conv2D(16, kernel_size=3, strides=2, padding='same',
                   kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(LeakyReLU(alpha=0.2))
    cnn.add(Dropout(0.5))

    cnn.add(Conv2D(32, kernel_size=3, strides=1, padding='same',
                   kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())
    cnn.add(LeakyReLU(alpha=0.2))
    cnn.add(Dropout(0.5))

    cnn.add(Conv2D(64, kernel_size=3, strides=2, padding='same',
                   kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())
    cnn.add(LeakyReLU(alpha=0.2))
    cnn.add(Dropout(0.5))

    cnn.add(Conv2D(128, kernel_size=3, strides=1, padding='same',
                   kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())
    cnn.add(LeakyReLU(alpha=0.2))
    cnn.add(Dropout(0.5))

    cnn.add(Conv2D(256, kernel_size=3, strides=2, padding='same',
                   kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())
    cnn.add(LeakyReLU(alpha=0.2))
    cnn.add(Dropout(0.5))

    cnn.add(Conv2D(512, kernel_size=3, strides=1, padding='same',
                   kernel_initializer='glorot_normal', bias_initializer='Zeros'))
    cnn.add(BatchNormalization())
    cnn.add(LeakyReLU(alpha=0.2))
    cnn.add(Dropout(0.5))

    cnn.add(Flatten())

    cnn.add(MinibatchDiscrimination(50, 30))

    image = Input(shape=(3, 32, 32))

    features = cnn(image)

    # first output (name=generation) is whether or not the discriminator
    # thinks the image that is being shown is fake, and the second output
    # (name=auxiliary) is the class that the discriminator thinks the image
    # belongs to.
    fake = Dense(1, activation='sigmoid', name='generation',
                 kernel_initializer='glorot_normal', bias_initializer='Zeros')(features)
    aux = Dense(class_num, activation='softmax', name='auxiliary',
                kernel_initializer='glorot_normal', bias_initializer= Constant(bias_value))(features)

    return Model(image, [fake, aux])


In [9]:
if __name__ == '__main__':

    # batch and latent size taken from the paper
    nb_epochs = 151
    batch_size = 100
    latent_size = 110
    global_acc = -100
    # Adam parameters suggested in https://arxiv.org/abs/1511.06434
    adam_lr = 0.0002
    adam_beta_1 = 0.5

    drop_list = [0, 1 , 2 , 3 , 4 , 5 , 6 ]
    drop_percentage = 0.99
    
   
    X_train, y_train, X_test, y_test, inverse_class_weights = get_imbalanced_train_data(drop_list, drop_percentage)
    mean_image = np.mean(X_train, axis=0)

    # build the discriminator, Choose Adam as optimizer according to GANHACK
    discriminator = build_discriminator()
    discriminator.compile(
        optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'categorical_crossentropy']
    )
    generator = build_generator(latent_size)

    latent = Input(shape=(latent_size,))
    image_class = Input(shape=(10,))

    # get a fake image
    fake = generator([latent, image_class])

    # we only want to be able to train generator for the combined model
    discriminator.trainable = False
    fake, aux = discriminator(fake)
    combined = Model([latent, image_class], [fake, aux])

    combined.compile(
        optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'categorical_crossentropy']
    )

    
    nb_train, nb_test = X_train.shape[0], X_test.shape[0]

    train_history = defaultdict(list)
    test_history = defaultdict(list)
    classifier_history = defaultdict(list)

    if load_weight:
        generator.load_weights(Path + '/params_generator_epoch_{0:03d}.hdf5'.format(load_epoch))
        discriminator.load_weights(Path + '/params_discriminator_epoch_{0:03d}.hdf5'.format(load_epoch))
    else:
        load_epoch = 0

    # load json and create model
    json_file = open('model_num.json', 'r')

    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights('model_num.h5')
    print("Loaded model from disk")

    loaded_model.save('model_num.hdf5')
    loaded_model=load_model('model_num.hdf5')


    for epoch in range(nb_epochs):
        print('Epoch {} of {}'.format(load_epoch + 1, nb_epochs))
        load_epoch += 1
        nb_batches = int(X_train.shape[0] / batch_size)
        progress_bar = Progbar(target=nb_batches)

        epoch_gen_loss = []
        epoch_disc_loss = []

        for index in range(nb_batches):
            progress_bar.update(index)
            # generate a new batch of noise
            noise = np.random.normal(0, 0.5, (batch_size, latent_size))

            # get a batch of real images
            image_batch = X_train[index * batch_size:(index + 1) * batch_size]
            label_batch = y_train[index * batch_size:(index + 1) * batch_size]
            
         
            #sample some labels from p_c
            sampled_labels = np.random.randint(0, class_num, batch_size)

            # generate a batch of fake images, using the generated labels as a
            # conditioner. We reshape the sampled labels to be
            # (batch_size, 1) so that we can feed them into the embedding
            # layer as a length one sequence
            generated_images = generator.predict(
                [noise, to_categorical(sampled_labels , class_num)], verbose=0)
            

            disc_real_weight = [np.ones(batch_size), 2 * np.ones(batch_size)]
            disc_fake_weight = [np.ones(batch_size), np.zeros(batch_size)]

            # According to GANHACK, We training our ACGAN-CIFAR10 in Real->D, Fake->D,
            # Noise->G, rather than traditional method: [Real, Fake]->D, Noise->G, actully,
            # it really make sense!

            for train_ix in range(3):
                if index % 30 != 0:
                    X_real = image_batch
                    # Label Soomthing
                    y_real = np.random.uniform(0.7, 1.2, size=(batch_size,))
                    aux_y1 = to_categorical(label_batch , class_num)

                 

                    epoch_disc_loss.append(discriminator.train_on_batch(X_real, [y_real, aux_y1]))
                    # Label Soomthing
                    X_fake = generated_images
                    y_fake = np.random.uniform(0.0, 0.3, size=(batch_size,))
                    aux_y2 = to_categorical(sampled_labels , class_num)


                    # see if the discriminator can figure itself out...
                    epoch_disc_loss.append(discriminator.train_on_batch(X_fake, [y_fake, aux_y2]))
                else:
                    # make the labels the noisy for the discriminator: occasionally flip the labels
                    # when training the discriminator
                    X_real = image_batch
                    y_real = np.random.uniform(0.0, 0.3, size=(batch_size,))
                    aux_y1 = to_categorical(label_batch , class_num)

                    epoch_disc_loss.append(discriminator.train_on_batch(X_real, [y_real, aux_y1]))
                    # Label Soomthing
                    X_fake = generated_images
                    y_fake = np.random.uniform(0.7, 1.2, size=(batch_size,))
                    aux_y2 = to_categorical(sampled_labels , class_num)

                    # see if the discriminator can figure itself out...
                    epoch_disc_loss.append(discriminator.train_on_batch(X_fake, [y_fake, aux_y2]))
            # make new noise. we generate Guassian Noise rather than Uniform Noise according to GANHACK
            noise = np.random.normal(0, 0.5, (2 * batch_size, latent_size))
            sampled_labels = np.random.randint(0, class_num, 2 * batch_size)

            sampled_labels = to_categorical(sampled_labels , class_num)

            # we want to train the generator to trick the discriminator
            # For the generator, we want all the {fake, not-fake} labels to say
            # not-fake
            trick = np.random.uniform(0.7, 1.2, size=(2 * batch_size,))

            epoch_gen_loss.append(combined.train_on_batch(
                [noise, sampled_labels], [trick, sampled_labels]))
            
        

        print('\nTesting for epoch {}:'.format(load_epoch))

        # evaluate the testing loss here

        # generate a new batch of noise
        noise = np.random.normal(0, 0.5, (nb_test, latent_size))

        # sample some labels from p_c and generate images from them
        sampled_labels = np.random.randint(0, class_num, nb_test)
        sampled_labels = to_categorical(sampled_labels , class_num)

        generated_images = generator.predict(
            [noise, sampled_labels], verbose=False)

        X = np.concatenate((X_test, generated_images))
        y = np.array([1] * nb_test + [0] * nb_test)
        aux_y = np.concatenate((to_categorical(y_test , class_num), sampled_labels), axis=0)

        # see if the discriminator can figure itself out...
        discriminator_test_loss = discriminator.evaluate(
            X, [y, aux_y], verbose=False)

        pred_y , pred_aux_y = discriminator.predict( X )

        precision , recall , f1 , _  = precision_recall_fscore_support(aux_y.argmax(axis = -1 ) , pred_aux_y.argmax(axis = -1) , average = "weighted" )

        test_history['precision'].append(precision)
        test_history['recall'].append(recall)
        test_history['f1'].append(f1)

        

        discriminator_train_loss = np.mean(np.array(epoch_disc_loss), axis=0)

        # make new noise
        noise = np.random.normal(0, 0.5, (2 * nb_test, latent_size))
        sampled_labels = np.random.randint(0, class_num, 2 * nb_test)

        sampled_labels = to_categorical(sampled_labels , class_num)

        trick = np.ones(2 * nb_test)

        generator_test_loss = combined.evaluate(
            [noise, sampled_labels],
            [trick,sampled_labels], verbose=False)

        generator_train_loss = np.mean(np.array(epoch_gen_loss), axis=0)

        # generate an epoch report on performance
        train_history['generator'].append(generator_train_loss)
        train_history['discriminator'].append(discriminator_train_loss)

        test_history['generator'].append(generator_test_loss)
        test_history['discriminator'].append(discriminator_test_loss)

        print('{0:<22s} | {1:4s} | {2:15s} | {3:5s}'.format(
            'component', *discriminator.metrics_names))
        print('-' * 65)

        ROW_FMT = '{0:<22s} | {1:<4.2f} | {2:<15.2f} | {3:<5.2f}'
        print(ROW_FMT.format('generator (train)',
                             *train_history['generator'][-1]))
        print(ROW_FMT.format('generator (test)',
                            *test_history['generator'][-1]))
        print(ROW_FMT.format('discriminator (train)',
                             *train_history['discriminator'][-1]))
        print(ROW_FMT.format('discriminator (test)',
                             *test_history['discriminator'][-1]))

        # save weights every epoch
                # generate some pictures to display
        noise = np.random.normal(0, 0.5, (100, latent_size))
        print ("Noise shape:" + str(noise.shape))
        sampled_labels = []
        
        for idx in range(10):
            for jdx in range(10):
                sampled_labels.append(idx)
                
        sampled_labels = np.asarray(sampled_labels)
      
        print(sampled_labels.shape)
        sampled_labels  = to_categorical(sampled_labels , class_num)
        generated_images = generator.predict([noise, sampled_labels])
        generated_images = generated_images.transpose(0, 2, 3, 1)
        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))
        
        
        
        # generating samples for testing 
        
        noise = np.random.normal(0, 0.5, (1000, latent_size))
            
        sampled_labels = []
        
        for idx in range(10):
            for jdx in range(100):
                sampled_labels.append(idx)
                
        sampled_labels = np.asarray(sampled_labels)
               
        print(sampled_labels.shape)
        sampled_labels  = to_categorical(sampled_labels , class_num)
        generated_imgs = generator.predict([noise, sampled_labels])
        generated_imgs = generated_imgs.transpose(0, 2, 3, 1)
        generated_imgs = np.asarray((generated_imgs * 127.5 + 127.5).astype(np.uint8))
        

        
        

        X_test_class = generated_imgs
        
        X_test_class = X_test_class.astype('float32')
        X_test-=mean_image
        X_test_class/= 128.
        y_pred = loaded_model.predict(X_test_class)

        prediction = y_pred.argmax(axis=-1)
        cnt_class = 0

        for idx in range(sampled_labels.shape[0]):
            if prediction[idx] == sampled_labels[idx].argmax(axis = -1):
                 cnt_class+=1


        acc = cnt_class/sampled_labels.shape[0]
        print(acc)
        
        # Calculating imbalance class accuracy only 
        X_temp = X_test_class[:200]
        y_pred_temp = loaded_model.predict(X_test_class)
        prediction_temp = y_pred_temp.argmax(axis = -1)
        cnt_class = 0
        
        for idx in range(X_temp.shape[0]):
            if prediction_temp[idx] == sampled_labels[idx].argmax(axis = -1):
                cnt_class+=1
        
        imb_acc = cnt_class/X_temp.shape[0]
        print("imbalance_acc --> " + str(imb_acc))
        
        
        def vis_square(data, padsize=1, padval=0):

            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data


        img = vis_square(generated_images)
        classifier_history['imb_accuracy'].append(imb_acc)
        if global_acc < acc:
            print("global_accuracy ---> " + str(global_acc))
            global_acc = max(global_acc , acc)
            classifier_history['accuracy'].append(global_acc)
            np.save(Path + "/generated_imgs" , generated_imgs)
            Image.fromarray(img).save(Path + '/plot_epoch_{0:03d}_generated.png'.format(load_epoch))
           


        pickle.dump({'train': train_history, 'test': test_history , 'classifier' : classifier_history},
                        open(Path + '/acgan-hisory.pkl', 'wb'))
        
 

Per class count before dropping: 
[5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]


W0820 08:46:42.785733 140065304487680 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 08:46:42.804075 140065304487680 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 08:46:42.806578 140065304487680 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0820 08:46:42.815986 140065304487680 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0820 08:46:42.827424 14

Per class count after dropping: 
[51, 51, 51, 51, 51, 51, 51, 5000, 5000, 5000]


W0820 08:46:44.352089 140065304487680 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0820 08:46:44.386986 140065304487680 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0820 08:46:45.444242 140065304487680 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0820 08:46:45.455428 140065304487680 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/nn_impl.py:180

<class 'tensorflow.python.framework.ops.Tensor'>


...

  1/153 [..............................] - ETA: 25:01

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


152/153 [============================>.] - ETA: 0s
Testing for epoch 1:


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 3.45 | 0.97            | 2.48 
generator (test)       | 2.95 | 0.68            | 2.27 
discriminator (train)  | 2.39 | 0.54            | 1.84 
discriminator (test)   | 4.29 | 0.50            | 3.79 
Noise shape:(100, 110)
(100,)
(1000,)
0.068
imbalance_acc --> 0.0
global_accuracy ---> -100
Epoch 2 of 151
152/153 [============================>.] - ETA: 0s
Testing for epoch 2:
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 2.97 | 1.05            | 1.92 
generator (test)       | 2.66 | 0.05            | 2.61 
discriminator (train)  | 1.86 | 0.44            | 1.43 
discriminator (test)   | 5.32 | 1.77            | 3.55 
Noise shape:(100, 110)
(100,)
(1000,)
0.123
imbalance_acc --> 0.005
global_accuracy ---> 0.068
Epoch 3 of 151
1

In [10]:
generator.save_weights( Path + 
            '/params_generator_epoch_{0:03d}.hdf5'.format(load_epoch), True)
discriminator.save_weights( Path +
            '/params_discriminator_epoch_{0:03d}.hdf5'.format(load_epoch), True)